In [0]:
!pip install geopandas

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!ls -la gdrive

In [0]:
import geopandas
import matplotlib
from shapely.geometry import Polygon, box

# counties = geopandas.read_file("../jsonFiles/gz_2010_us_050_00_20m.json", encoding='latin-1')
counties = geopandas.read_file("gdrive/My Drive/counties.json")
# counties[counties['STATE'].isin(['02','15']) == False].plot(figsize=(30,20), color='#3B3C6E');
# counties.plot(figsize=(30,20))
classAcounties = counties[counties['STATE'].isin(['02','15']) == False]
classAcounties.plot(figsize=(30,20))

In [0]:
from shapely.geometry import Polygon, box
import json
import progressbar

#### plot map
# classAcounties.plot(figsize=(30,20))

baseX = -110    #-125
endX = -100     #-65
stepX = 3
lenX = int((endX - baseX) * stepX)
baseY = 35      #25
endY = 40       #50
stepY = 3
lenY = int((endY - baseY) * stepY)

progressBarWidget = [progressbar.Percentage(),
' ',
progressbar.Bar('=', '[', ']'),
' '
]
progressBar = progressbar.ProgressBar(maxval=lenX*lenY, widgets=progressBarWidget)
progressBar.start()

grid = []
polygon_grid = []
for i in range(lenX):
  grid_row = []
  polygon_grid_row = []
  for j in range(lenY):
    cell_data = []
    polygon_cell = box(baseX + i/stepX, baseY + j/stepY,baseX + (i+1)/stepX, baseY + (j+1)/stepY)

    #### plot grid cells
    # x,y = cell_data['polygon'].exterior.xy
    # matplotlib.pyplot.plot(x,y)

    #### calculate counties intersections
    for index, row in counties.iterrows():
      countyArea = row['geometry'].area
      intersectionArea = polygon_cell.intersection(row['geometry']).area
      if intersectionArea != 0:
        fips = int(row['STATE'] + row['COUNTY'], 10)
        cell_data.append({'fips':fips, 'percent':intersectionArea/countyArea})

    grid_row.append(cell_data)
    polygon_grid_row.append(polygon_cell)
    progressBar.update((i * lenY) + j)
  grid.append(grid_row)
  polygon_grid.append(polygon_grid_row)

progressBar.finish()
x = json.dumps(grid)

with open('gdrive/My Drive/map_intersection.json', 'w') as fd:
  fd.write(x)

100% [=======================================================================] 
